# Business Understanding

Our company, Microsoft, wants to be able to factor in sentiments about our products that are expressed on social media platforms, namely X (formerly Twitter), where millions of users share positive, negative, or neutral sentiments about products they use in an organic way. While consumer or user reviews left through official platforms such as Yelp or Amazon are important to consider, the pool of users is rather limited as not everyone will go the extra mile to provide a review through those "official" channels, and taking more organic feedback expressed through platforms like X into consideration can provide us an extra layer of insight as to how the public is interacting with and perceiving our products. 

Specifically, Microsoft wants to be able to filter tweets carrying Positive sentiments towards our products or services so that we are able to assess what's working well and what areas can and should be developed further.

The way we will go about this, is to create a **binary classifier model** that is trained and tested on a dataset containing tweets where users expressed various emotions or reactions to other products, as the essential issue is how well our model is able to parse through and differentiate between Positive tweets on the one hand, and Negative or Neutral tweets on the other hand.

In evaluating the efficacy of our model, we need to ascertain how effective the model was at correctly identifying Positive tweets. Put differently, we need to keep an eye out for **False Positives**, since our model would cause many issues for us if it incorrectly pulled in Negative or Neutral tweets, which would complicate or muddy our analysis. While False Negatives are also something to consider, they are of less importance as missing out on Positive tweets would be less damaging to us as it is generally better to avoid overestimating Positive feedback anyway.

# Data Understanding

As this problem is about analyzing and categorizing sentiments expressed through text, we will need to build a model capable of Natural Language Processing, or NLP for short. This model needs to be adept at processing and parsing through text, and categorizing the text as 'Positive', 'Not Positive' (ie: 'Negative' or 'Neutral').

We will engage in NLP to build a binary classifier that is capable of differentiating between 'Positive' and 'Negative' or 'Neutral' sentiments.

### Dataset
For these purposes, we will work with a [dataset we retrieved from data.world](https://data.world/crowdflower/brands-and-product-emotions) that contains more than 9,000 tweets expressing Positive, Negative, or Neutral sentiments towards Apple or Google products.

The dataset contains information across three columns:
1. `tweets_text`, which contains the text of the collected tweets themselves.
2. `emotion_in_tweet_is_directed_at`, which indicates which product the tweet is speaking to. This column contains a number of values, however they all fall under either Apple of Google products (with the exception of a number of rows that contain 'No data').
3. `is_there_an_emotion_directed_at_a_brand_or_product`, which categorizes the collected tweets according to either 'Positive emotion', 'Negative emotion', or 'No emotion toward brand or product'. There is a fourth category, 'I can't tell', which we need to investigate more before deciding whether or not to remove these entries.

### Features and Labels
The `tweets_text` column will serve as our features, or X, while `is_there_an_emotion_directed_at_a_brand_or_product` will serve as our labels, or y. 

To be more specific, we will use the `tweets_text` column to generate **TF-IDF (Term Frequency-Inverse Document Frequency)** scores, which assigns numeric values for key terms by weighing their frequency within a certain text against their frequency across different texts. This will help our model in gaining signals from significant words and reduce noise from frequent, insignificant words. These will be our features, at least in the initial baseline model.

The second column, `emotion_in_tweet_is_directed_at`, is not relevant for our purposes, as our task is to build a model that can categorize the sentiments expressed, and not to determine *whom* the sentiments are addressing.

### Class Imbalance
Our labels have significant class imbalance, with Negative sentiments only comprising 6% of the data compared to 33% for Positive and 59% for Neutral. Since we are adding the Negative and Neutral tweets together in a single class, this gives us:
- 33% Positive
- 65% Not Positive
  
This will cause issues for us in both training the model and evaluating its performance according to success metrics.

To compensate for this imbalance we will deploy **Class Weighting**, which is a strategy that can be used with certain models that gives higher importance to minority classes. 

### Success Metrics
As described above, we need to pay extra mind to our model's ability to correctly identify Positive tweets, placing a higher importance on the rates of False Positives. However, we should also keep an eye on False Negatives to minimize the number of Positive tweets falling through the cracks.

For this reason, we will rely on the following metrics:
1. **Precision Score**, which evaluates how accurate we were in actually identifying Positive tweets (telling us the rates of False Positives). The higher the precision score, the lower the rate of False Positives. This will be our primary metric.
2. **F1-Score**, which weighs the rate of False Positives and False Negatives, since we also want to minimize the rate at which we misclassify Positive tweets as Not Positive. This will be a seconadry metric for us.

### Model Selection
We will start with a simple baseline model as an initial performance check before moving on to more complex models. Since we have some class imbalance, we will deploy **Multinomial Naive Bayes (Multinomial NB)**, which can help compensate for this imbalance using a weighted approach. This model is also better at producing the metrics we outlined above. 

Multinomial NB will require us to compute **TF-IDF (Term Frequency-Inverse Document Frequency)**, as described above. However, this will have limited applicability on future application of the model, as future tweets will undoubtedly contain new slang and terms that arise and will not be computed in this current model.

After establishing a baseline, we will then move on to testing out more complex models, namely **BERTweet (Bidirectional Encoder Representations from Transformers)**, a Deep Learning model which is well-suited for analyzing tweets in particular as it is trained on 850 million English tweets and can process special characters such as emojies, hashtags, etc., and is capable of determining contextual meaning from limited text as tweets had a limit of 140 characters back in 2013, which is when our dataset was compiled. 

## Data Preparation

Since we want to first run a baseline MultinomialNB model, we will need to carry out several steps of Data Preparation in order for the data to be ready for modeling. 

In [2]:
# importing necessary libraries

import re
import torch
from torch.utils.data import DataLoader
import pandas as pd
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import precision_recall_fscore_support
from datasets import Dataset
from tqdm import tqdm
import transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification, \
                         AdamW
transformers.logging.set_verbosity_error()

import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('punkt', quiet=True)
nltk.download('wordnet', quiet=True)

import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

In [3]:
# loading the data
# specifying the encoding as the file is a non-UTF-8 CSV file

df = pd.read_csv("data/judge_tweet_product.csv", encoding="latin-1")

df.head()

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion


As discussed earlier, the `emotion_in_tweet_is_directed_at` column is not necessary for our purposes of determining the sentiment of the tweet, so we will drop it from our DataFrame.

In [4]:
# dropping the second column

df = df.drop('emotion_in_tweet_is_directed_at', axis=1)


In [5]:
# checking if dropping column worked

df.head()

,tweet_text,is_there_an_emotion_directed_at_a_brand_or_product
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Positive emotion


#### Now we need to check for any null values.

In [6]:
df.isna().sum()

tweet_text                                            1
is_there_an_emotion_directed_at_a_brand_or_product    0
dtype: int64

We have one row with null values. Let's take a look to see if we're able to simply drop it.

In [7]:
df[df.isna().any(axis=1)]

,tweet_text,is_there_an_emotion_directed_at_a_brand_or_product
6,NaN,No emotion toward brand or product


In [8]:
# dropping the row with null value

df = df.dropna()

# making sure it worked

df.isna().sum()

tweet_text                                            0
is_there_an_emotion_directed_at_a_brand_or_product    0
dtype: int64

#### Cleaning Target Values

In [9]:
df['is_there_an_emotion_directed_at_a_brand_or_product'].value_counts()

is_there_an_emotion_directed_at_a_brand_or_product
No emotion toward brand or product    5388
Positive emotion                      2978
Negative emotion                       570
I can't tell                           156
Name: count, dtype: int64

There is a fourth category in our Target column, **'I can't tell'**, indicating that those who gathered this dataset were unsure about the sentiment expressed in about 2% of the tweets. 

Let's take a quick look. Since they only make up 2% of the dataset, it may be best to simply drop these entries and focus on our analysis on the Positive, Negative, and Neutral categories.

In [10]:
df[df['is_there_an_emotion_directed_at_a_brand_or_product'] == "I can't tell"]

,tweet_text,is_there_an_emotion_directed_at_a_brand_or_product
90,Thanks to @mention for publishing the news of ...,I can't tell
102,ÛÏ@mention &quot;Apple has opened a pop-up st...,I can't tell
237,Just what America needs. RT @mention Google to...,I can't tell
341,The queue at the Apple Store in Austin is FOUR...,I can't tell
368,Hope it's better than wave RT @mention Buzz is...,I can't tell
...,...,...
9020,It's funny watching a room full of people hold...,I can't tell
9032,"@mention yeah, we have @mention , Google has n...",I can't tell
9037,"@mention Yes, the Google presentation was not ...",I can't tell
9058,&quot;Do you know what Apple is really good at...,I can't tell


In [11]:
# removing "I can't tell" entries in the Target column from the dataframe

df_clean = df[df['is_there_an_emotion_directed_at_a_brand_or_product'] != "I can't tell"]

In [12]:
df_clean

,tweet_text,is_there_an_emotion_directed_at_a_brand_or_product
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Positive emotion
...,...,...
9088,Ipad everywhere. #SXSW {link},Positive emotion
9089,"Wave, buzz... RT @mention We interrupt your re...",No emotion toward brand or product
9090,"Google's Zeiger, a physician never reported po...",No emotion toward brand or product
9091,Some Verizon iPhone customers complained their...,No emotion toward brand or product


In [13]:
# checking to see if it worked

df_clean['is_there_an_emotion_directed_at_a_brand_or_product'].value_counts()

is_there_an_emotion_directed_at_a_brand_or_product
No emotion toward brand or product    5388
Positive emotion                      2978
Negative emotion                       570
Name: count, dtype: int64

Since our Target column contains three categories ("Positive emotion, "Negative emotion", "No emotion toward brand or product"), and we'll need to collapse the latter two labels into a single category in order for us to conduct binary classification.

We'll do this by converting the label values into 1s and 0s.
- `1` meaning Positive
- `0` meaning Not Positive

In [14]:
# mapping 1s and 0s onto the Target column values

df_clean.loc[:,'is_there_an_emotion_directed_at_a_brand_or_product'] = df['is_there_an_emotion_directed_at_a_brand_or_product'].map({
    'Positive emotion': 1,
    'Negative emotion': 0,
    'No emotion toward brand or product': 0
})

# converting binary values into integers

df_clean.loc[:,'is_there_an_emotion_directed_at_a_brand_or_product'] = \
df_clean['is_there_an_emotion_directed_at_a_brand_or_product'].astype(int)

In [15]:
# making sure it worked

df_clean['is_there_an_emotion_directed_at_a_brand_or_product'].unique()

array([0, 1], dtype=object)

For convenience, we'll change the name of our Target column to something shorter.

In [16]:
df_clean = df_clean.rename(columns={'is_there_an_emotion_directed_at_a_brand_or_product': 'is_positive'})

# checking if it worked

df_clean.head()

,tweet_text,is_positive
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,0
1,@jessedee Know about @fludapp ? Awesome iPad/i...,1
2,@swonderlin Can not wait for #iPad 2 also. The...,1
3,@sxsw I hope this year's festival isn't as cra...,0
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,1


#### Train-Test-Splitting
Now that our dataset is cleaned, we'll split the data into training and test sets before preprocessing the text data in order to prevent data leakage.

In [17]:
# setting X (Features) and y (Target)

X = df_clean[['tweet_text']]
y = df_clean['is_positive']

# splitting the data

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.25,
    random_state=42
)

#### Creating Custom Transformers and Pipelines
##### **Baseline Model**

We can now start preparing our X variable (features) for our modeling.

We'll need to convert all words to lowercase, remove special characters, remove stop words, tokenize the text and apply lemmatization in order for our text to be processed uniformly across all variations of the same words.

These steps can be built into our Pipeline, which we will deploy to prevent data leakage.

First, we need to create a **custom transformer** to carry out these steps, before creating the Pipeline.

In [18]:
# defining custom transformer with parameters that allow it to work like an sklearn transformer inside the pipelines

class TextPreprocessor(BaseEstimator, TransformerMixin):
    """
    This custom transformer will
    1) Lower case the text,
    2) Remove special characters,
    3) Tokenize the text,
    4) Lemmatize the text
    """
    # initializing Lemmatizer for later use
    def __init__(self):
        self.lemmatizer = WordNetLemmatizer()

    # fitting the transformer
    def fit(self, X, y=None):
        return self

    # transforming the text, creating list to store preprocessed words
    def transform(self, X):
        preprocessed_words = []

        # for loop to transform all words in the text
        for word in X:
            
            word = word.lower() # lowercasing the words
            
            word = re.sub(r'[^a-z\s]', '', word) # removing special characters
            
            tokens = nltk.word_tokenize(word) # tokenizing words
            
            lemmatized_words = ' '.join([self.lemmatizer.lemmatize(word) for word in tokens]) # lemmatizing the tokens \
                                                                                              # into a list of strings
            
            preprocessed_words.append(lemmatized_words) # adding preprocessed words back to the list
            
        return preprocessed_words



Now that we have our custom function ready, we can prepare a Pipeline for our baseline MultinomialNB model. 

This Pipeline will cover the following steps:
1. Run the Custom Transformer we just created.
2. Vectorize the text into TF-IDF features.
3. Run a MultinomialNB model.

In [19]:
# creating Pipeline for MultinomialNB modeling

baseline_pipe = Pipeline([
    ('preprocessor', TextPreprocessor()), # preprocessing transformer
    ('vectorizer', TfidfVectorizer()), # Convert to TF-IDF features
    ('classifier', MultinomialNB()), # Run the MultinomialNB model
])

##### **BERTweet Model**
Next, we need to prepare a Pipeline for our BERTweet model, which is a more advanced model that we will use after establishing a baseline.

As discussed above, BERTweet is a more advanced, pre-trained model that is adept at handling tweets specifically, and preprocessing text for this model is different than for other types of models. Since the model has its own tokenizer, we need to keep the text as raw as possible and don't need to tokenize, remove special characters, lemmatize or stem the text, or lowercase the text.

First, we need to create two Custom Transformers tailored for our BERTweet model:
1. Tokenizer Transformer
2. Classifier Transformer

Since BERTweet is a deep learning model, we will need to integrate **PyTorch Tensors** into our preprocessing transformers, as they are multi-dimensional arrays similar to NumPy arrays but more are optmized for deep learning.

In [20]:
# creating Custom Tokenizer Transformer for BERTweet

class BERTTokenizer(BaseEstimator, TransformerMixin):
    def __init__(self, model_name="bert-base-uncased", max_length=128):
        self.model_name = model_name
        self.max_length = max_length
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_name, use_fast=True)


    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return self.__call__(X)

    def __call__(self, texts):
        if not isinstance(texts, list):
            texts = [texts]
    
        return self.tokenizer(
            text=texts,
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors='pt'
        )

In [21]:
# creating Custom Classifier Transformer for BERTweet

class BERTClassifier(BaseEstimator):
    """
    Custom Classifier specifically tailored for BERTweet sentiment analysis
    1) load pretrained BERTweet model
    2) run inference without training (training is not required)
    3) convert model outputs into our class labels ('Positive' or 'Not Positive')
    4) retun probabilities or confidence scores for our predicted classifications
    """

    def __init__(self, model_name = "bert-base-uncased", num_labels = 2, device = None):
        self.model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)
        self.device = torch.device(('mps' if torch.has_mps else 'cpu'), dtype=torch.float16) if device is None else device
        self.model.to(self.device, dtype=torch.float16)
        self.model.eval() # setting the model to evaluate

    def fit(self, X, y=None):
        return self # pretrained model, so no training needed

    def predict(self, X):
        """
        use tokenized inputs to generate sentiment classification predictions
        """
        with torch.no_grad(): # disabling gradient calculation for quicker inference
            outputs = self.model(**X) # passing tokenized inputs into BERT model
        preds = torch.argmax(outputs.logits, dim=1).cpu().numpy() # converting predictions to NumPy
        return preds # returning classification labels

    def predict_proba(self, X):
        """
        finding prediction probabilities
        """
        with torch.set_grad_enabled(False):
            outputs = self.model(**X)
        probs = torch.nn.functional.softmax(outputs.logits, dim=1).cpu().numpy()
        return probs # returning classification probabilities

The model we invoked in the transformers above, `'bert-base-uncased'`, requires fine-tuning on data before it can successfully be deployed in a Pipeline.

We'll also need to create validation sets for this portion, and then convert the sets into a `Hugging Face` format in order to be compatible with the trainer we will use.

In [22]:
# splitting training sets into training and validation sets

X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train,
    test_size=0.2, random_state=42
)

In [23]:
# creating copies of training and validation sets to keep originals for baseline model

X_train_copy, y_train_copy = X_train.copy(), y_train.copy()
X_val_copy, y_val_copy = X_val.copy(), y_val.copy()

X_train_copy, y_train_copy = X_train_copy.reset_index(drop=True), y_train_copy.reset_index(drop=True)
X_val_copy, y_val_copy = X_val_copy.reset_index(drop=True), y_val_copy.reset_index(drop=True)


In [24]:
class PTDataset(Dataset):
    """
    creating a custom transformer to tokenize text 
    and convert into PyTorch tensors
    """
    def __init__(self, texts, labels, tokenizer):
        # Ensure texts is a list of strings, not a list of lists
        self.texts = [str(text) for text in (texts.tolist() if hasattr(texts, "tolist") else texts)]
        self.labels = labels.tolist() if hasattr(labels, "tolist") else labels
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        if isinstance(idx, list):  # Handling batch sampling
            texts = [self.texts[i] for i in idx]
            labels = [self.labels[i] for i in idx]
        else:
            texts = [self.texts[idx]]
            labels = [self.labels[idx]]
    
        tokenized = self.tokenizer.transform(texts)
    
        return {
            "input_ids": tokenized["input_ids"],  
            "attention_mask": tokenized["attention_mask"],
            "labels": torch.tensor(labels, dtype=torch.long),
        }








In [25]:
tokenizer = BERTTokenizer()

# converting lists to PyTorch datasets
train_dataset = PTDataset(
    X_train_copy.squeeze().tolist(),
    y_train_copy.tolist(),
    tokenizer
)

val_dataset = PTDataset(
    X_val_copy.squeeze().tolist(),
    y_val_copy.tolist(),
    tokenizer
)

# creating PyTorch DataLoaders
train_dataloader = DataLoader(
    train_dataset,
    batch_size=4,
    shuffle=True # training should be randomized so shuffle=True
)

val_dataloader = DataLoader(
    val_dataset,
    batch_size=4,
    shuffle=False # no need to shuffle validation data
)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [26]:
# manually fine-tuning the 'bert-base-uncased' model using PyTorch

model = AutoModelForSequenceClassification.from_pretrained( # loading the model
    'bert-base-uncased', 
    num_labels=2
) 

device = torch.device('mps' if torch.backends.mps.is_built() else 'cpu') # optimizing for Apple Metal hardware
model.to(device, dtype=torch.float16)

optimizer = AdamW(model.parameters(), lr=2e-5) # defining the optimizer

# defining the loss function
class_weights = torch.tensor([1.0, 3.0], device=device)  # adjust the weight of positive class
criterion = torch.nn.CrossEntropyLoss(weight=class_weights)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [ ]:
epochs = 5

for e in range(epochs):
    model.train()
    total_loss = 0

    loop = tqdm(
        train_dataloader, 
        desc=f'Epoch {e+1}/{epochs}'
    )    

    for batch in loop:
        batch = {key: val.to(device) for key, val in batch.items()} # moving batch to new device
        
        # forward pass
        outputs = model(**batch)
        loss = criterion(outputs.logits, batch['labels'])

        # backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        loop.set_postfix(loss=loss.item())

    avg_train_loss = total_loss / len(train_dataloader)
    print(f'Epoch {e+1} - Average Loss: {avg_train_loss:.4f}')

print('Training Complete')

Epoch 1/5:   0%|                                                                                                                                      | 0/1341 [00:00<?, ?it/s]

Now that we have run the training loop on our training dataset, we need to evaluate the model's performance on a validation dataset in order to check for overfitting.

In [231]:
model.eval()  # setting to evaluation mode
y_true = []
y_pred = []
total_loss = 0

with torch.no_grad():  
    for batch in val_dataloader:
        batch = {key: val.to(device) for key, val in batch.items()}  
        
        outputs = model(**batch)
        loss = criterion(outputs.logits, batch['labels'])
        total_loss += loss.item()

        preds = torch.argmax(outputs.logits, dim=1)
        y_true.extend(batch['labels'].cpu().numpy())
        y_pred.extend(preds.cpu().numpy())

avg_val_loss = total_loss / len(val_dataloader)

# Compute Precision, Recall, F1 Score
precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='binary', zero_division=0)

# Compute Accuracy
accuracy = np.sum(np.array(y_pred) == np.array(y_true)) / len(y_true)

print(f"Validation Loss: {avg_val_loss:.4f}, Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}")


Validation Loss: 0.6557, Accuracy: 0.6401
Precision: 0.0000, Recall: 0.0000, F1 Score: 0.0000


With our custom transformers made, we can go ahead and create our BERT Pipeline.

In [27]:
# creating pipeline for BERTweet model

bert_pipe = Pipeline([
    ('tokenizer', BERTTokenizer()),
    ('classifier', BERTClassifier())
])

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/bertweet-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [24]:
from transformers import AutoModelForSequenceClassification

model_name = "vinai/bertweet-base"
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
print("Model loaded successfully!")


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/bertweet-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model loaded successfully!


# Exploratory Data Analysis

# Conclusion

## Limitations

## Recommendations

## Next Steps